# **Chapter 5.오차역전파법**
# **5.4 단순한 계층 구현하기**
# **5.5 활성화 함수 계층 구현하기**
# **5.6 Affine/Softmax 계층 구현하기**
# **5.7 오차역전파법 구현하기**

## 곱셈 계층 : 역전파법에서 x,y 바꿔서 곱함

In [1]:
class MulLayer:
    def __init__(self):
        self.x=None
        self.y=None

    def forward(self,x,y):
        self.x=x
        self.y=y
        out=x*y

        return out

    def backward(self,dout):
        dx=dout*self.y
        dy=dout*self.x

        return dx,dy

In [2]:
apple=100
apple_num=2
tax=1.1

mul_apple_layer = MulLayer()
mul_tax_layer=MulLayer()

## 순전파법

In [3]:
apple_price = mul_apple_layer.forward(apple,apple_num)
price = mul_tax_layer.forward(apple_price,tax)

print(price)

220.00000000000003


## 역전파법

In [4]:
dprice=1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple,dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple,dapple_num,dtax)

2.2 110.00000000000001 200


## 덧셈 계층 : 미분하면 1

In [5]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self,x,y):
        out=x+y

        return out

    def backward(self,dout):
        dx=dout*1
        dy=dout*1

        return dx,dy

In [6]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)  # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)  # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price = mul_tax_layer.forward(all_price, tax)  # (4)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # (1)

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)

price: 715
dApple: 2.2
dApple_num: 110
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650


# ReLu 계층

In [7]:
class Relu:
    def __init__(self):
        self.mask=None

    def forward(self,x):
        self.mask=(x<=0)
        out = x.copy()
        out[self.mask]=0
        return out

    def backward(self,dout):
        dout[self.mask] = 0
        dx=dout
        return dx

In [8]:
import numpy as np
x = np.array([[1.0,1.5],[-2.0,3.0]])
print(x)
mask=(x<=0)
print(mask)

[[ 1.   1.5]
 [-2.   3. ]]
[[False False]
 [ True False]]


# Sigmoid 계층

In [9]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

# Affine 계층 (행렬)

In [10]:
X=np.random.rand(2)
W=np.random.rand(2,3)
B=np.random.rand(3)

print(X.shape)
print(W.shape)
print(B.shape)

Y=np.dot(X,W)+B

(2,)
(2, 3)
(3,)


In [11]:
X_dot_W = np.array([[0,0,0],[10,10,10]])
B = np.array([1,2,3])
print(X_dot_W+B)

[[ 1  2  3]
 [11 12 13]]


In [12]:
dY = np.array([[1,2,3],[4,5,6]])
dB = np.sum(dY,axis=0)
print(dB)

[5 7 9]


In [13]:
class Affine:
    def __init__(self,W,b):
        self.W=W
        self.b=b
        self.x=None
        self.dW=None
        self.db=None

    def forward(self,x):
        self.x=x
        out=np.dot(x,self.W)+self.b

        return out

    def backward(self,dout):
        dx = np.dot(dout,self.W.T)
        self.dW = np.dot(self.x.T,dout)
        self.db = np.sum(dout,axis=0)

        return dx

# Softmax-with-Loss 계층

softmax 함수 : exp(입력값) / exp(입력값) 전체의 합  
softmax(zi): i번째 클래스에 대한 확률 출력  
모든 출력값의 합은 1 (확률 분포의 성질)  
입력값 중 큰 값일수록 더 높은 확률로 변환됨  
출력값은 0~1 사이의 값  
소프트맥스 계층을 구현할 때 손실 함수인 교차 엔트로피 오차 포함하여 Softmax-with-Loss 계층이라는 이름으로 구현

In [14]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss=None
        self.y=None
        self.t=None

    def forward(self,x,t):
        self.t=t
        self.y=softmax(x)
        self.loss=cross_entropy_error(self.y,self.t)
        return self.loss

    def backward(self,dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y-self.t)/batch_size

        return dx

# 오차역전파법을 적용한 신경망 구현
1. 미니배치 : 훈련 데이터 중 일부를 무작위로 가져온다. 이 미니배치의 손실 함수 값을 줄이는 것이 목표  
2. 기울기 산출 : 미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기 구함. (여기서 오차역전파법 사용)  
3. 매개변수 갱신  
4. 반복  

In [15]:
class Relu:
    def __init__(self):
        self.mask=None

    def forward(self,x):
        self.mask=(x<=0)
        out = x.copy()
        out[self.mask]=0
        return out

    def backward(self,dout):
        dout[self.mask] = 0
        dx=dout
        return dx


class Affine:
    def __init__(self,W,b):
        self.W=W
        self.b=b
        self.x=None
        self.dW=None
        self.db=None

    def forward(self,x):
        self.x=x
        out=np.dot(x,self.W)+self.b

        return out

    def backward(self,dout):
        dx = np.dot(dout,self.W.T)
        self.dW = np.dot(self.x.T,dout)
        self.db = np.sum(dout,axis=0)

        return dx


class SoftmaxWithLoss:
    def __init__(self):
        self.loss=None
        self.y=None
        self.t=None

    def forward(self,x,t):
        self.t=t
        self.y=softmax(x)
        self.loss=cross_entropy_error(self.y,self.t)
        return self.loss

    def backward(self,dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y-self.t)/batch_size

        return dx

In [16]:
import sys, os
import numpy as np
from common.layers import Affine, Relu, SoftmaxWithLoss
from common.gradient import numerical_gradient
# 파이썬에서 OrderedDict 자료구조 사용 (입력된 순서를 기억하는 딕셔너리)
from collections import OrderedDict


class TwoLayerNet:
# 구조: (Affine1 → ReLU) → (Affine2 → SoftmaxWithLoss)
    
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1) # 확률이 가장 큰 y값 선택
        if t.ndim != 1 : t = np.argmax(t, axis=1) # 가장 큰 t값 선택
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

In [22]:
import sys, os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist

(x_train, t_train),(x_test,t_test) = load_mnist(normalize=True,one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch,t_batch)
grad_backprop = network.gradient(x_batch,t_batch)

for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key]-grad_numerical[key]))
    print(key + ":" + str(diff))

W1:7.555367395781029e-08
b1:8.60824698151806e-07
W2:7.17901771718426e-09
b2:1.4060312683306097e-07
